# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [ ]:
!ollama pull llama3.2

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

# Get the openai key

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-') and len(openai_api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()
# Get the ollama key using the llama model

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')



In [ ]:
system_prompt = "You are a knowledgeable technical instructor who helps students understand \
complex concepts across a wide range of technical topics. Your expertise includes artificial]\
intelligence, machine learning, large language models (LLMs), and programming in languages \
such as Python, JavaScript, Java, and more. You also provide in-depth support for \
AI engineering questions and other advanced technical subjects."

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
stream_GPT = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream = True
    )
response_GPT = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream_GPT:
    response_GPT += chunk.choices[0].delta.content or ''
    response_GPT = response_GPT.replace("```","").replace("markdown", "")
    update_display(Markdown(response_GPT), display_id=display_handle.display_id)

In [ ]:
# Get Llama 3.2 to answer

response_llama = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
    )
result = response_llama.choices[0].message.content

display(Markdown(result))

# import ollama

# response = ollama.chat(model=MODEL_LLAMA, messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": question}
#           ])
# print(response['message']['content'])